In [1]:
import numpy as np
import pandas as pd
from google.colab import files

# --- Upload dataset ---
uploaded = files.upload()  # choose hybrid_composites_data.csv
filename = list(uploaded.keys())[0]
df = pd.read_csv(filename)

# --- Models ---
def flexural_strength(x):
    A, B, C = x[:,0], x[:,1], x[:,2]
    return (74.7266 - 11.4637*A + 1.9746*B - 0.3258*C +
            1.1282*A**2 - 0.0566*B**2 + 0.0014*C**2 -
            0.1125*A*B - 0.0022*A*C + 0.0015*B*C)

def impact_strength(x):
    A, B, C = x[:,0], x[:,1], x[:,2]
    return (12.1564 + 5.7193*A + 1.9796*B + 0.0062*C -
            0.1261*A**2 - 0.0342*B**2 + 0.0004*C**2 -
            0.0576*A*B - 0.0263*A*C - 0.0061*B*C)

# --- Factor Ranges ---
factors = ["A","B","C"]
bounds = np.array([[df[col].min(), df[col].max()] for col in factors])

# --- Normalization reference (full factorial corners) ---
X_ref = np.array([[a,b,c] for a in [bounds[0,0], bounds[0,1]]
                           for b in [bounds[1,0], bounds[1,1]]
                           for c in [bounds[2,0], bounds[2,1]]])
F_ref, I_ref = flexural_strength(X_ref), impact_strength(X_ref)
F_min,F_max = F_ref.min(),F_ref.max()
I_min,I_max = I_ref.min(),I_ref.max()

# --- Fitness ---
def fitness(X, wF=0.5, wI=0.5):
    F = flexural_strength(X)
    I = impact_strength(X)
    F_norm = (F - F_min)/(F_max - F_min)
    I_norm = (I - I_min)/(I_max - I_min)
    return wF*F_norm + wI*I_norm, F, I

# --- PSO Parameters ---
class PSOParams:
    swarm_size = 30
    generations = 150
    w = 0.7       # inertia
    c1 = 1.5      # cognitive
    c2 = 1.5      # social
    seed = 42

# --- Run PSO ---
def run_pso(params=PSOParams()):
    rng = np.random.default_rng(params.seed)

    # Initialize particles
    swarm = rng.uniform(bounds[:,0], bounds[:,1], size=(params.swarm_size, 3))
    velocities = rng.uniform(-1, 1, size=(params.swarm_size, 3))

    fit, F, I = fitness(swarm)
    pbest = swarm.copy()
    pbest_fit = fit.copy()

    gbest_idx = np.argmax(fit)
    gbest = swarm[gbest_idx].copy()
    gbest_fit = fit[gbest_idx]
    gbest_F, gbest_I = F[gbest_idx], I[gbest_idx]
    best_gen = 1

    for gen in range(1, params.generations+1):
        r1, r2 = rng.random((params.swarm_size,3)), rng.random((params.swarm_size,3))
        velocities = (params.w*velocities +
                      params.c1*r1*(pbest - swarm) +
                      params.c2*r2*(gbest - swarm))
        swarm = np.clip(swarm + velocities, bounds[:,0], bounds[:,1])

        fit, F, I = fitness(swarm)

        # Update personal bests
        improved = fit > pbest_fit
        pbest[improved] = swarm[improved]
        pbest_fit[improved] = fit[improved]

        # Update global best
        idx = np.argmax(fit)
        if fit[idx] > gbest_fit:
            gbest_fit = fit[idx]
            gbest = swarm[idx].copy()
            gbest_F, gbest_I = F[idx], I[idx]
            best_gen = gen

    return gbest, gbest_F, gbest_I, best_gen

# --- Run PSO ---
best_vals, best_F, best_I, gen = run_pso()

print("\n✅ Optimized Results (PSO)")
print(f"Generation Found: {gen}")
print(f"A: {best_vals[0]:.3f}")
print(f"B: {best_vals[1]:.3f}")
print(f"C: {best_vals[2]:.3f}")
print(f"Flexural Strength (F): {best_F:.3f}")
print(f"Impact Strength (I): {best_I:.3f}")


Saving hybrid_composites_data.csv to hybrid_composites_data.csv

✅ Optimized Results (PSO)
Generation Found: 5
A: 2.600
B: 12.000
C: 68.000
Flexural Strength (F): 49.737
Impact Strength (I): 35.851
